In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


In [2]:
# restore all of our data structures
import pickle
data = pickle.load( open( "../data/training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('../data/intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
model = tflearn.DNN(net)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [4]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [5]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [6]:
# load our saved model
model.load('../data/model.tflearn')

INFO:tensorflow:Restoring parameters from /Users/tuhin.sharma/Documents/Git/chatbot-incident-ticket/chatbot_platform/data/model.tflearn


In [7]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return random.choice(i['responses'])
    
            results.pop(0)

In [8]:
st = "my macbook is restarting everytime I try to open MS office"
print(classify(st))
print(response(st))

[('thanks', 0.73998564), ('incident', 0.2593052)]
See you later, Thanks for visiting


In [9]:
st = "Please add me to our Ivanti Corporate Dropbox instance as I need to access the shared contents for our APAC team."
print(classify(st))
print(response(st))

[('incident', 0.9993455)]
I see you are facing a problem. Tell me more about it. But before that let me suggest some solutions that you can try.


In [10]:
st = "Good day"
print(classify(st))
print(response(st))

[('thanks', 0.9162025)]
Have a nice day


In [11]:
st = "what do you do in your free time?"
print(classify(st))
print(response(st))

[('profile', 0.3866273), ('greeting', 0.35607383)]
Why don't you stick to asking me IT related problems.


In [12]:
st = "what are your interests"
print(classify(st))
print(response(st))

[('profile', 0.8517136)]
Please do not ask me personal questions


In [13]:
st = "are you afraid of something?"
print(classify(st))
print(response(st))

[('emotional', 0.77460945)]
I am not an emotional person. In fact I don't know if I am a person at all.


In [14]:
st = "I love you"
print(classify(st))
print(response(st))

[('emotional', 0.8253065)]
I am not an emotional person. In fact I don't know if I am a person at all.


In [15]:
st = "are you interested in india pakistan war"
print(classify(st))
print(response(st))

[('history', 0.39262554)]
You should probably ask Herodotus, the great Historian


In [16]:
st = "thanks"
print(classify(st))
print(response(st))

[('thanks', 0.9501769)]
My pleasure


In [17]:
st = "good evening"
print(classify(st))
print(response(st))

[('thanks', 0.87734556)]
My pleasure


In [18]:
st = "do you know who won the world war 2?"
print(classify(st))
print(response(st))

[('profile', 0.5927058), ('history', 0.28464857)]
Please do not ask me personal questions


In [19]:
st = "my phone is not responding?"
print(classify(st))
print(response(st))

[('incident', 0.64765567), ('thanks', 0.35231474)]
I see you are facing a problem. Tell me more about it. But before that let me suggest some solutions that you can try.


In [20]:
st = "who was mahatma gandhi?"
print(classify(st))
print(response(st))

[('politics', 0.65069616)]
I am not sure if I am interested in that.
